In [1]:
import sys
print(sys.executable)

/home/hillenr/dev/ml/.venv/bin/python


In [2]:
import torch
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Device:", torch.cuda.get_device_name(0))
    print("Compute capability:", torch.cuda.get_device_capability(0))
    print("CUDA build:", torch.version.cuda)
    print("cuDNN:", torch.backends.cudnn.version())

Torch: 2.6.0+cu124
CUDA available: True
Device: NVIDIA GeForce RTX 2060 with Max-Q Design
Compute capability: (7, 5)
CUDA build: 12.4
cuDNN: 90100


In [18]:
# Download and build Sudoku dataset
!python dataset/build_sudoku_dataset.py --output-dir data/sudoku-extreme-1k-aug-1000  --subsample-size 1000 --num-aug 1000

train.csv: 100%|██████████████████████████████| 719M/719M [00:07<00:00, 101MB/s]
100%|███████████████████████████████████████| 1000/1000 [01:47<00:00,  9.32it/s]
test.csv: 100%|████████████████████████████| 79.4M/79.4M [00:01<00:00, 53.3MB/s]
100%|██████████████████████████████| 422786/422786 [00:00<00:00, 2004964.67it/s]


In [20]:
# ---------------------------
# Pretty printing utilities
# ---------------------------
def pretty_print_sudoku(grid: np.ndarray):
    if isinstance(grid, torch.Tensor):
        grid = grid.detach().cpu().numpy()
    grid = grid.astype(int)
    for i in range(9):
        if i % 3 == 0 and i != 0:
            print("- - - - - - - - - - -")
        row = []
        for j in range(9):
            if j % 3 == 0 and j != 0:
                row.append("|")
            row.append(str(grid[i, j]-1))
        print(" ".join(row))

In [22]:
import numpy as np
import os

for dir in ["data/sudoku-extreme-1k-aug-1000/train/", "data/sudoku-extreme-1k-aug-1000/test/"]:
    print(dir)
    with os.scandir(dir) as entries:
        for entry in entries:
            if entry.is_file():  # Check if the entry is a file (not a subdirectory)
                if entry.name.endswith(".npy"):
                    array = np.load(dir + entry.name, allow_pickle=True)
                    print(f"{entry.name} - ndim: {array.ndim}, shape: {array.shape}")
                    if array.ndim == 2:
                        pretty_print_sudoku(array[0,:].reshape(9,9))
    print()

data/sudoku-extreme-1k-aug-1000/train/
all__puzzle_identifiers.npy - ndim: 1, shape: (1001000,)
all__group_indices.npy - ndim: 1, shape: (1001,)
all__labels.npy - ndim: 2, shape: (1001000, 81)
1 8 5 | 2 7 3 | 4 9 6
7 9 6 | 5 8 4 | 3 1 2
2 3 4 | 6 9 1 | 5 8 7
- - - - - - - - - - -
6 2 9 | 4 5 8 | 7 3 1
8 5 7 | 1 3 9 | 2 6 4
4 1 3 | 7 2 6 | 8 5 9
- - - - - - - - - - -
9 4 2 | 3 1 5 | 6 7 8
5 7 1 | 8 6 2 | 9 4 3
3 6 8 | 9 4 7 | 1 2 5
all__puzzle_indices.npy - ndim: 1, shape: (1001001,)
all__inputs.npy - ndim: 2, shape: (1001000, 81)
0 0 0 | 0 0 3 | 0 9 0
0 9 6 | 0 0 4 | 0 0 0
2 0 0 | 0 0 0 | 5 0 0
- - - - - - - - - - -
0 0 9 | 4 5 0 | 0 3 0
0 5 0 | 1 3 0 | 2 0 4
0 0 0 | 0 2 0 | 0 0 9
- - - - - - - - - - -
0 0 0 | 0 1 0 | 6 0 8
5 7 0 | 0 0 2 | 0 0 0
0 0 0 | 9 0 0 | 0 0 0

data/sudoku-extreme-1k-aug-1000/test/
all__puzzle_identifiers.npy - ndim: 1, shape: (422786,)
all__group_indices.npy - ndim: 1, shape: (422787,)
all__labels.npy - ndim: 2, shape: (422786, 81)
5 9 8 | 4 6 1 | 2 7 3
7 3 1 |

In [19]:
# Sudoku 9x9 Extreme (1k)
!hf download sapientinc/HRM-checkpoint-sudoku-extreme --repo-type model --local-dir checkpoints/sudoku

Fetching 3 files:   0%|                                   | 0/3 [00:00<?, ?it/s]Downloading 'all_config.yaml' to 'checkpoints/sudoku/.cache/huggingface/download/G-AS8IAG4FreZDu1U09ksncp270=.c1247b366c02ae2b01f1f8c8308b60e85b5ea894.incomplete'

all_config.yaml: 100%|█████████████████████████| 854/854 [00:00<00:00, 2.17MB/s]
Download complete. Moving file to checkpoints/sudoku/all_config.yaml

.gitattributes: 1.57kB [00:00, 1.37MB/s]
Download complete. Moving file to checkpoints/sudoku/.gitattributes
Fetching 3 files:  33%|█████████                  | 1/3 [00:00<00:00,  2.88it/s]Downloading 'checkpoint' to 'checkpoints/sudoku/.cache/huggingface/download/TCtVUeJteg8BwkIET3RKU6YwXzc=.0befd4a9f33a090d46df00728160ddbd69b81bcfdb8064ccfde0c867dd3836b9.incomplete'

checkpoint:   0%|                                    | 0.00/109M [00:00<?, ?B/s]
checkpoint:  39%|██████████▍                | 42.1M/109M [00:03<00:05, 11.8MB/s]
checkpoint: 100%|████████████████████████████| 109M/109M [00:03<00:00, 

In [22]:
# Example: evaluate Sudoku 1k
!python evaluate.py checkpoint="checkpoints/sudoku/checkpoint"

Starting evaluation
W0827 21:28:45.358000 28352 torch/_inductor/utils.py:1137] [0/0] Not enough SMs to use max_autotune_gemm mode
/home/hillenr/dev/ml/.venv/lib64/python3.12/site-packages/torch/_inductor/compile_fx.py:1948: UserWarning: NVIDIA GeForce RTX 2060 with Max-Q Design does not support bfloat16 compilation natively, skipping
  warnings.warn(
/home/hillenr/dev/ml/.venv/lib64/python3.12/site-packages/torch/_inductor/compile_fx.py:1948: UserWarning: NVIDIA GeForce RTX 2060 with Max-Q Design does not support bfloat16 compilation natively, skipping
  warnings.warn(
/home/hillenr/dev/ml/.venv/lib64/python3.12/site-packages/torch/_inductor/compile_fx.py:1948: UserWarning: NVIDIA GeForce RTX 2060 with Max-Q Design does not support bfloat16 compilation natively, skipping
  warnings.warn(
/home/hillenr/dev/ml/.venv/lib64/python3.12/site-packages/torch/_inductor/compile_fx.py:1948: UserWarning: NVIDIA GeForce RTX 2060 with Max-Q Design does not support bfloat16 compilation natively, skip

In [24]:
import json, yaml, random
from typing import Optional, Tuple, Dict, Any
from utils.functions import load_model_class
# ---------------------------
# Dataset loading
# ---------------------------
def load_sudoku_metadata(data_path: str) -> Dict[str, Any]:
    test_dir = os.path.join(data_path, "test")
    with open(os.path.join(test_dir, "dataset.json"), "r") as f:
        return json.load(f)


def load_random_sudoku_example(data_path: str, puzzle_index: Optional[int] = None) -> Tuple[Dict[str, torch.Tensor], int]:
    """Loads a single example from the Sudoku test set as a batch of size 1."""
    test_dir = os.path.join(data_path, "test")
    with open(os.path.join(test_dir, "dataset.json"), "r") as f:
        meta = json.load(f)

    # Sudoku builder uses a single set named "all"
    assert len(meta["sets"]) == 1, "Sudoku test set is expected to have a single subset"
    set_name = meta["sets"][0]

    inputs = np.load(os.path.join(test_dir, f"{set_name}__inputs.npy"))
    labels = np.load(os.path.join(test_dir, f"{set_name}__labels.npy"))
    # For Sudoku, puzzle_identifiers is all zeros
    puzzle_identifiers = np.zeros((inputs.shape[0],), dtype=np.int32)

    assert inputs.shape[1] == 81 and labels.shape[1] == 81, "Sudoku expects 9x9 (seq_len=81)"

    n = inputs.shape[0]
    if puzzle_index is None:
        puzzle_index = random.randrange(n)

    batch = {
        "inputs": torch.from_numpy(inputs[puzzle_index:puzzle_index + 1].astype(np.int32)),
        "labels": torch.from_numpy(labels[puzzle_index:puzzle_index + 1].astype(np.int32)),
        "puzzle_identifiers": torch.from_numpy(puzzle_identifiers[puzzle_index:puzzle_index + 1]),
    }
    return batch, puzzle_index


# ---------------------------
# Checkpoint/model loading
# ---------------------------
def load_model_from_checkpoint(
    *,
    device: torch.device,
    data_meta: Dict[str, Any],
    checkpoint_dir: Optional[str] = None,
    ckpt_path: Optional[str] = None,
    hf_repo: str = "sapientinc/HRM-checkpoint-sudoku-extreme",
    hf_filename: str = "model.pt",
):
    """Builds the model and loads weights from either a local training directory or HF."""
    model = None

    def _normalize_state_keys(state: Dict[str, torch.Tensor]) -> Dict[str, torch.Tensor]:
        # Handle torch.compile wrapper and loss-head wrapper prefixes
        def strip_prefix(k: str, p: str) -> str:
            return k[len(p):] if k.startswith(p) else k

        out = {}
        for k, v in state.items():
            k2 = strip_prefix(k, "_orig_mod.")
            k2 = strip_prefix(k2, "model.")
            out[k2] = v
        return out

    # Case A: explicit checkpoint file path
    if ckpt_path is not None:
        if not os.path.exists(ckpt_path):
            raise FileNotFoundError(f"Checkpoint file not found: {ckpt_path}")

        obj = torch.load(ckpt_path, map_location=device)

        # Packaged format: contains cfg + model_state_dict
        if isinstance(obj, dict) and ("cfg" in obj) and ("model_state_dict" in obj):
            cfg = obj["cfg"]
            if hasattr(cfg, "to_container"):
                arch_cfg = dict(cfg.arch)  # type: ignore
            else:
                arch_cfg = dict(cfg["arch"])  # type: ignore

            model_cls = load_model_class(arch_cfg["name"])  # type: ignore
            model = model_cls(arch_cfg).to(device)
            # Be lenient about prefixes in packaged files
            state = obj["model_state_dict"]  # type: ignore
            if isinstance(state, dict):
                state = _normalize_state_keys(state)
            model.load_state_dict(state)  # type: ignore
            model.eval()
            return model, arch_cfg

        # Raw state_dict format: need YAML config nearby (in same dir or checkpoint_dir)
        cfg_dir = os.path.dirname(ckpt_path)
        cfg_path = os.path.join(cfg_dir, "all_config.yaml")
        if (checkpoint_dir is not None) and (not os.path.exists(cfg_path)):
            cfg_path = os.path.join(checkpoint_dir, "all_config.yaml")
        if yaml is None:
            raise RuntimeError("PyYAML is required to load local checkpoints. Please install pyyaml.")
        assert os.path.exists(cfg_path), f"Missing config: {cfg_path}"

        with open(cfg_path, "r") as f:
            cfg = yaml.safe_load(f)
        arch_cfg = cfg["arch"]

        model_cfg = dict(
            **{k: v for k, v in arch_cfg.items() if k != "name"},
            batch_size=1,
            seq_len=data_meta["seq_len"],
            vocab_size=data_meta["vocab_size"],
            num_puzzle_identifiers=data_meta["num_puzzle_identifiers"],
        )
        model_cls = load_model_class(arch_cfg["name"])  # e.g., "hrm.hrm_act_v1@HierarchicalReasoningModel_ACTV1"
        model = model_cls(model_cfg).to(device)

        state = obj if isinstance(obj, dict) else obj.state_dict()
        state = _normalize_state_keys(state)
        model.load_state_dict(state, assign=True)
        model.eval()
        return model, arch_cfg

    if checkpoint_dir is not None:
        # Expect a training directory with all_config.yaml and a step_* file
        if yaml is None:
            raise RuntimeError("PyYAML is required to load local checkpoints. Please install pyyaml.")

        cfg_path = os.path.join(checkpoint_dir, "all_config.yaml")
        assert os.path.exists(cfg_path), f"Missing config: {cfg_path}"

        with open(cfg_path, "r") as f:
            cfg = yaml.safe_load(f)

        arch_cfg = cfg["arch"]

        # Build model config using dataset-derived fields
        model_cfg = dict(
            **{k: v for k, v in arch_cfg.items() if k != "name"},
            batch_size=1,
            seq_len=data_meta["seq_len"],
            vocab_size=data_meta["vocab_size"],
            num_puzzle_identifiers=data_meta["num_puzzle_identifiers"],
        )

        model_cls = load_model_class(arch_cfg["name"])  # e.g., "hrm.hrm_act_v1@HierarchicalReasoningModel_ACTV1"
        model = model_cls(model_cfg).to(device)

        # Pick the latest step_* file if not specified
        step_files = [f for f in os.listdir(checkpoint_dir) if f.startswith("step_") and "." not in f]
        assert len(step_files), f"No step_* files found in {checkpoint_dir}"
        step_files.sort(key=lambda s: int(s.split("_")[1]))
        ckpt_path = os.path.join(checkpoint_dir, step_files[-1])

        state = torch.load(ckpt_path, map_location=device)
        state = _normalize_state_keys(state)
        model.load_state_dict(state, assign=True)

        model.eval()
        return model, arch_cfg

In [27]:

data_path: str = "data/sudoku-extreme-1k-aug-1000"
checkpoint_dir: Optional[str] = None
ckpt_path: Optional[str] = "checkpoints/sudoku/checkpoint"
hf_repo: str = "sapientinc/HRM-checkpoint-sudoku-extreme"
hf_filename: str = "model.pt",
puzzle_index: Optional[int] = None
seed: int = 320

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

if not os.path.exists(os.path.join(data_path, "test", "dataset.json")):
    raise FileNotFoundError(
        f"Dataset not found at {data_path}. Build it via dataset/build_sudoku_dataset.py or adjust data_path."
    )

meta = load_sudoku_metadata(data_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model, arch_cfg = load_model_from_checkpoint(
    device=device,
    data_meta=meta,
    checkpoint_dir=checkpoint_dir,
    ckpt_path=ckpt_path,
    hf_repo=hf_repo,
    hf_filename=hf_filename,
)

print("Loaded model on", device)
print(model)
print("halt_max_steps:", arch_cfg.get("halt_max_steps"))

batch, chosen_index = load_random_sudoku_example(data_path, puzzle_index=puzzle_index)
# Move tensors to device
batch = {k: v.to(device) for k, v in batch.items()}

print(f"Selected puzzle index: {chosen_index}")
print("Input puzzle (0 means blank):")
# Tokens are 1..10 (PAD=0). Subtract 1 for human-readable 0..9.
pretty_print_sudoku((batch["inputs"].view(9, 9)))

# Step-by-step execution
# Ensure carry tensors are allocated on the same device as the model
with torch.device("cuda" if device.type == "cuda" else "cpu"):
    carry = model.initial_carry(batch)
final_outputs = None


Loaded model on cuda
HierarchicalReasoningModel_ACTV1(
  (inner): HierarchicalReasoningModel_ACTV1_Inner(
    (embed_tokens): CastedEmbedding()
    (lm_head): CastedLinear()
    (q_head): CastedLinear()
    (puzzle_emb): CastedSparseEmbedding()
    (rotary_emb): RotaryEmbedding()
    (H_level): HierarchicalReasoningModel_ACTV1ReasoningModule(
      (layers): ModuleList(
        (0-3): 4 x HierarchicalReasoningModel_ACTV1Block(
          (self_attn): Attention(
            (qkv_proj): CastedLinear()
            (o_proj): CastedLinear()
          )
          (mlp): SwiGLU(
            (gate_up_proj): CastedLinear()
            (down_proj): CastedLinear()
          )
        )
      )
    )
    (L_level): HierarchicalReasoningModel_ACTV1ReasoningModule(
      (layers): ModuleList(
        (0-3): 4 x HierarchicalReasoningModel_ACTV1Block(
          (self_attn): Attention(
            (qkv_proj): CastedLinear()
            (o_proj): CastedLinear()
          )
          (mlp): SwiGLU(
      

In [34]:
print(batch["inputs"].shape)
print(batch["labels"].shape)
print(batch["puzzle_identifiers"])

torch.Size([1, 81])
torch.Size([1, 81])
tensor([0], device='cuda:0', dtype=torch.int32)


In [35]:
print(meta)

{'pad_id': 0, 'ignore_label_id': 0, 'blank_identifier_id': 0, 'vocab_size': 11, 'seq_len': 81, 'num_puzzle_identifiers': 1, 'total_groups': 422786, 'mean_puzzle_examples': 1.0, 'sets': ['all']}


In [38]:
import wandb
wb_token="d473a6902feca68e46d1adb185325ade4b9bf0d8"
wandb.login(key=wb_token)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /home/hillenr/.netrc
wandb: Currently logged in as: hillenr (hillenr-home) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [51]:
!OMP_NUM_THREADS=8 python pretrain.py data_path=data/sudoku-extreme-1k-aug-1000 epochs=4000 eval_interval=2000 global_batch_size=200 lr=7e-5 puzzle_emb_lr=7e-5 weight_decay=1.0 puzzle_emb_weight_decay=1.0

  0%|                                                 | 0/20000 [00:00<?, ?it/s]wandb: Currently logged in as: hillenr (hillenr-home) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.21.1
wandb: Run data is saved locally in /home/hillenr/dev/ml/HRM/wandb/run-20250831_102438-czygs0ed
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run HierarchicalReasoningModel_ACTV1 judicious-dinosaur
wandb: ⭐️ View project at https://wandb.ai/hillenr-home/Sudoku-extreme-1k-aug-1000%20ACT-torch
wandb: 🚀 View run at https://wandb.ai/hillenr-home/Sudoku-extreme-1k-aug-1000%20ACT-torch/runs/czygs0ed
[Rank 0, World Size 1]: Epoch 0
 50%|████████████████▌                | 10000/20000 [1:56:06<1:33:18,  1.79it/s]W0831 12:20:55.742000 128295 torch/_inductor/utils.py:1137] [0/1] Not enough SMs to use max_autotune_gemm mode
[Rank 0, World Size 1]: Epoch 2000
100%|███████████████████████████████████| 20000/20000 [5:28:24<00:00,  1.81it/